# Notebook Title

## Setup Python and R environment
you can ignore this section

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [6]:
#Motor vehicle collisions 2023, dropped rows without lat/long as there were 7300 rows and I dont have API credits https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95/about_data
import pandas as pd

df = pd.read_csv('df_cleaned.csv')

In [7]:
df =df.sample(10000)

In [8]:
print(df.dtypes)

CRASH DATE                        object
CRASH TIME                        object
BOROUGH                           object
ZIP CODE                         float64
LATITUDE                         float64
LONGITUDE                        float64
LOCATION                          object
ON STREET NAME                    object
CROSS STREET NAME                 object
OFF STREET NAME                   object
NUMBER OF PERSONS INJURED          int64
NUMBER OF PERSONS KILLED           int64
NUMBER OF PEDESTRIANS INJURED      int64
NUMBER OF PEDESTRIANS KILLED       int64
NUMBER OF CYCLIST INJURED          int64
NUMBER OF CYCLIST KILLED           int64
NUMBER OF MOTORIST INJURED         int64
NUMBER OF MOTORIST KILLED          int64
CONTRIBUTING FACTOR VEHICLE 1     object
CONTRIBUTING FACTOR VEHICLE 2     object
CONTRIBUTING FACTOR VEHICLE 3     object
CONTRIBUTING FACTOR VEHICLE 4     object
CONTRIBUTING FACTOR VEHICLE 5     object
COLLISION_ID                       int64
VEHICLE TYPE COD

## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

In [9]:
# Save the df_cleaned DataFrame as a CSV file
df.to_csv('df_cleaned2.csv', index=False)


## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [13]:
# Code adapted from:
# https://gis.stackexchange.com/questions/363830/applying-the-censusgeocode-package-to-an-entire-dataframe-of-geocoded-data
# Defines a geocode function that accepts lat/long and spits out geographies
# The code then runs that funciton in parllel (for speed).

import pandas as pd
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import glob
import json
import requests
import pandas as pd
from pprint import pprint
from tqdm import tqdm


import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df['LATITUDE']
        longitudes = df['LONGITUDE']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df)))

    #return pd.DataFrame(data)
        return pd.DataFrame([x for x in data if x])

census_geos_df = bulk_geocode(df['LATITUDE'], df['LONGITUDE']) 
census_geos_df.head()
len(census_geos_df)


  5%|▍         | 459/10000 [00:00<00:08, 1172.66it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'
Error geocoding (0.0, 0.0): 'Census Blocks'
Error geocoding (0.0, 0.0): 'Census Blocks'
Error geocoding (0.0, 0.0): 'Census Blocks'


  8%|▊         | 810/10000 [00:00<00:08, 1070.32it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'
Error geocoding (0.0, 0.0): 'Census Blocks'


 11%|█▏        | 1145/10000 [00:01<00:08, 1081.19it/s]

...

 96%|█████████▌| 9609/10000 [00:08<00:00, 1162.14it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'


100%|██████████| 10000/10000 [00:08<00:00, 1151.99it/s]

Error geocoding (0.0, 0.0): 'Census Blocks'


9961

In [14]:
census_geos_df

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,39,360810769021002,+40.7152988,1002,0,36,1002,210701006114638,BK,...,076902,-073.8333178,1,16299,11,-073.8333178,G5040,L,U,081
1,,119,360470458002000,+40.6256969,2000,0,36,2000,210701004657702,BK,...,045800,-073.9677874,2,21237,35,-073.9677874,G5040,L,U,047
2,,552,360810125002000,+40.7724818,2000,0,36,2000,210701006109175,BK,...,012500,-073.9176480,2,18017,244,-073.9176480,G5040,L,U,081
3,,1051,360050199004002,+40.8365484,4002,0,36,4002,210701006027606,BK,...,019900,-073.9280507,4,19844,401,-073.9280507,G5040,L,U,005
4,,65,360850170051019,+40.5397084,1019,0,36,1019,210701004607689,BK,...,017005,-074.1801431,1,25041,34,-074.1801431,G5040,L,U,085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9956,,0,360470085001000,+40.6776725,1000,0,36,1000,210701004651390,BK,...,008500,-074.0084680,1,39824,0,-074.0084680,G5040,L,U,047
9957,,225,360470119011009,+40.6724138,1009,0,36,1009,210701004649753,BK,...,011901,-073.9886116,1,18250,140,-073.9886116,G5040,L,U,047
9958,,599,360050425005000,+40.8723242,5000,0,36,5000,210701006027787,BK,...,042500,-073.8762752,5,22486,199,-073.8762752,G5040,L,U,005
9959,,165,360610303001000,+40.8693802,1000,0,36,1000,210701008620522,BK,...,030300,-073.9158884,1,7092,72,-073.9158884,G5040,L,U,061


## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [16]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,360810769021002,36,081,076902,1002
1,360470458002000,36,047,045800,2000
2,360810125002000,36,081,012500,2000
3,360050199004002,36,005,019900,4002
4,360850170051019,36,085,017005,1019
...,...,...,...,...,...
9956,360470085001000,36,047,008500,1000
9957,360470119011009,36,047,011901,1009
9958,360050425005000,36,005,042500,5000
9959,360610303001000,36,061,030300,1000


In [17]:
df_with_geos = pd.concat(
    [ 
        df.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,GEOID,STATE,COUNTY,TRACT,BLOCK
0,2023-08-08,17:11:00,QUEENS,11375.0,40.715120,-73.832240,"(40.71512, -73.83224)",QUEENS BOULEVARD,78 AVENUE,NaN,...,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,360810769021002,36,081,076902,1002
1,2023-10-26,18:00:00,NaN,NaN,40.624554,-73.967100,"(40.624554, -73.9671)",EAST 9 STREET,NaN,NaN,...,Sedan,Bus,NaN,NaN,NaN,360470458002000,36,047,045800,2000
2,2023-07-18,16:07:00,QUEENS,11102.0,40.772083,-73.917520,"(40.772083, -73.91752)",NaN,NaN,24-46 29 STREET,...,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN,360810125002000,36,081,012500,2000
3,2023-05-07,02:30:00,BRONX,10452.0,40.836320,-73.927480,"(40.83632, -73.92748)",NaN,NaN,1133 OGDEN AVENUE,...,Sedan,Sedan,Sedan,NaN,NaN,360050199004002,36,005,019900,4002
4,2023-01-22,03:47:00,NaN,NaN,40.540500,-74.177315,"(40.5405, -74.177315)",SNEDEN AVENUE,NaN,NaN,...,Sedan,Pick-up Truck,NaN,NaN,NaN,360850170051019,36,085,017005,1019


In [18]:
# Saving the DataFrame as a CSV file
df_with_geos.to_csv('geocoded_col_10k.csv', index=False)
